In [44]:
import joblib
import pandas as pd

In [45]:
transcription = joblib.load("output/transcription_google_colab.jl")
print(transcription[0].keys())
print(transcription[0]["segments"])

dict_keys(['text', 'segments', 'language'])
[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 1.0, 'text': ' Thank you all.', 'tokens': [50363, 6952, 345, 477, 13, 50413], 'temperature': 0.0, 'avg_logprob': -0.45480142982260696, 'compression_ratio': 1.611353711790393, 'no_speech_prob': 0.2748943865299225}, {'id': 1, 'seek': 0, 'start': 1.0, 'end': 2.0, 'text': ' Good afternoon.', 'tokens': [50413, 4599, 6672, 13, 50463], 'temperature': 0.0, 'avg_logprob': -0.45480142982260696, 'compression_ratio': 1.611353711790393, 'no_speech_prob': 0.2748943865299225}, {'id': 2, 'seek': 0, 'start': 2.0, 'end': 3.0, 'text': ' How are you guys doing?', 'tokens': [50463, 1374, 389, 345, 3730, 1804, 30, 50513], 'temperature': 0.0, 'avg_logprob': -0.45480142982260696, 'compression_ratio': 1.611353711790393, 'no_speech_prob': 0.2748943865299225}, {'id': 3, 'seek': 0, 'start': 3.0, 'end': 4.0, 'text': ' It looks fun.', 'tokens': [50513, 632, 3073, 1257, 13, 50563], 'temperature': 0.0, 'avg_logprob': -0.45480142982

In [46]:
subclip_0_df = pd.DataFrame(transcription[0]["segments"])
print(subclip_0_df.head())
for i in range(-9, 0):
    print(subclip_0_df["text"].iloc[i])

   id  seek  start  end                                            text   
0   0     0    0.0  1.0                                  Thank you all.  \
1   1     0    1.0  2.0                                 Good afternoon.   
2   2     0    2.0  3.0                         How are you guys doing?   
3   3     0    3.0  4.0                                   It looks fun.   
4   4     0    4.0  7.0   How many fans have done themselves last week?   

                                              tokens  temperature   
0                 [50363, 6952, 345, 477, 13, 50413]          0.0  \
1                     [50413, 4599, 6672, 13, 50463]          0.0   
2     [50463, 1374, 389, 345, 3730, 1804, 30, 50513]          0.0   
3                [50513, 632, 3073, 1257, 13, 50563]          0.0   
4  [50563, 1374, 867, 3296, 423, 1760, 2405, 938,...          0.0   

   avg_logprob  compression_ratio  no_speech_prob  
0    -0.454801           1.611354        0.274894  
1    -0.454801           1.611

In [47]:
subclip_1_df = pd.DataFrame(transcription[1]["segments"])
for i in range(10):
    print(subclip_1_df["text"].iloc[i])

 How do you think this position happens?
 So mRNA will have different nucleoside fields
 and I could have tRNA bringing in one amino acid at a time.
 How do I know how does the system,
 how can you encode for the fact that,
 if I hit this particular code off,
 I will bring in the correct amino acid.
 There's chemistry behind it.
 What is this position mechanism?
 Yeah.


In [48]:
subclips_list = joblib.load("output/subclips_list.jl")
subclip_info_df = pd.DataFrame(subclips_list)
subclip_info_df

,index,start,end,path
0,0,0,1128,output/subclip_0_0_1128.mp3
1,1,1108,2246,output/subclip_1_1108_2246.mp3
2,2,2226,3364,output/subclip_2_2226_3364.mp3
3,3,3344,4482,output/subclip_3_3344_4482.mp3
4,4,4462,5587,output/subclip_4_4462_5587.mp3


In [49]:
print(type(subclip_info_df["start"].iloc[0]))

<class 'numpy.int64'>


In [50]:
subclip_0_df["subclip_index"] = int(subclip_info_df["index"].iloc[0])
subclip_0_df["start"] += subclip_info_df["start"].iloc[0]
subclip_0_df["end"] += subclip_info_df["start"].iloc[0]

for i in range(1, len(transcription)):
    subclip_i_df = pd.DataFrame(transcription[i]["segments"])
    subclip_i_df["subclip_index"] = int(subclip_info_df["index"].iloc[i])
    subclip_i_df["start"] += subclip_info_df["start"].iloc[i]
    subclip_i_df["end"] += subclip_info_df["start"].iloc[i]
    subclip_0_df = pd.concat([subclip_0_df, subclip_i_df], ignore_index=True)
subclip_0_df

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,subclip_index
0,0,0,0.00,1.00,Thank you all.,"[50363, 6952, 345, 477, 13, 50413]",0.0,-0.454801,1.611354,0.274894,0
1,1,0,1.00,2.00,Good afternoon.,"[50413, 4599, 6672, 13, 50463]",0.0,-0.454801,1.611354,0.274894,0
2,2,0,2.00,3.00,How are you guys doing?,"[50463, 1374, 389, 345, 3730, 1804, 30, 50513]",0.0,-0.454801,1.611354,0.274894,0
3,3,0,3.00,4.00,It looks fun.,"[50513, 632, 3073, 1257, 13, 50563]",0.0,-0.454801,1.611354,0.274894,0
4,4,0,4.00,7.00,How many fans have done themselves last week?,"[50563, 1374, 867, 3296, 423, 1760, 2405, 938,...",0.0,-0.454801,1.611354,0.274894,0
...,...,...,...,...,...,...,...,...,...,...,...
1480,162,108472,5553.12,5558.56,"one more class to see you guys, but there wil...","[50683, 530, 517, 1398, 284, 766, 345, 3730, 1...",0.0,-0.208055,1.514894,0.001780,4
1481,163,108472,5558.56,5567.44,game-changing systems that you'll learn next ...,"[50955, 983, 12, 22954, 3341, 326, 345, 1183, ...",0.0,-0.208055,1.514894,0.001780,4
1482,164,108472,5567.44,5571.68,"If you have questions, bring to us. And if yo...","[51399, 1002, 345, 423, 2683, 11, 2222, 284, 5...",0.0,-0.208055,1.514894,0.001780,4
1483,165,110968,5571.68,5576.64,you will find problems as really quite straig...,"[50363, 345, 481, 1064, 2761, 355, 1107, 2407,...",0.0,-0.245308,1.240310,0.044770,4


In [51]:
subclip_0_df[133:145]

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,subclip_index
133,133,109600,1097.00,1106.00,I said coded manner because which amino acid ...,"[50413, 314, 531, 30817, 5642, 780, 543, 23206...",0.0,-0.154138,1.660232,0.238554,0
134,134,109600,1108.00,1125.00,How do you think this position happens? So mR...,"[50963, 1374, 466, 345, 892, 428, 2292, 4325, ...",0.0,-0.154138,1.660232,0.238554,0
135,0,0,1108.00,1110.24,How do you think this position happens?,"[50363, 1374, 466, 345, 892, 428, 2292, 4325, ...",0.0,-0.299951,1.581395,0.019813,1
136,1,0,1110.24,1112.96,So mRNA will have different nucleoside fields,"[50475, 1406, 47227, 481, 423, 1180, 17751, 41...",0.0,-0.299951,1.581395,0.019813,1
137,2,0,1112.96,1116.92,and I could have tRNA bringing in one amino a...,"[50611, 290, 314, 714, 423, 256, 27204, 6079, ...",0.0,-0.299951,1.581395,0.019813,1
138,3,0,1116.92,1119.02,"How do I know how does the system,","[50809, 1374, 466, 314, 760, 703, 857, 262, 10...",0.0,-0.299951,1.581395,0.019813,1
139,4,0,1119.02,1121.20,"how can you encode for the fact that,","[50914, 703, 460, 345, 37773, 329, 262, 1109, ...",0.0,-0.299951,1.581395,0.019813,1
140,5,0,1122.42,1124.52,"if I hit this particular code off,","[51084, 611, 314, 2277, 428, 1948, 2438, 572, ...",0.0,-0.299951,1.581395,0.019813,1
141,6,0,1124.52,1126.74,I will bring in the correct amino acid.,"[51189, 314, 481, 2222, 287, 262, 3376, 23206,...",0.0,-0.299951,1.581395,0.019813,1
142,7,0,1128.36,1129.60,There's chemistry behind it.,"[51381, 1318, 338, 16585, 2157, 340, 13, 51443]",0.0,-0.299951,1.581395,0.019813,1


In [52]:
subclip_0_df[530:545]

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,subclip_index
530,395,111420,2226.72,2230.44,"this five prime capping, this sort of a squar...","[50589, 428, 1936, 6994, 1275, 2105, 11, 428, ...",0.0,-0.220976,1.672489,0.004981,1
531,396,111420,2230.44,2232.52,and then poly-A tail tagging.,"[50775, 290, 788, 7514, 12, 32, 7894, 49620, 1...",0.0,-0.220976,1.672489,0.004981,1
532,397,111420,2232.52,2233.72,"So maybe on the footnote,","[50879, 1406, 3863, 319, 262, 38512, 11, 50939]",0.0,-0.220976,1.672489,0.004981,1
533,398,111420,2233.72,2236.32,"I mentioned five prime cap is a purple square,","[50939, 314, 4750, 1936, 6994, 1451, 318, 257,...",0.0,-0.220976,1.672489,0.004981,1
534,399,111420,2236.32,2238.68,"and then A is, if you see multiple A's,","[51069, 290, 788, 317, 318, 11, 611, 345, 766,...",0.0,-0.220976,1.672489,0.004981,1
535,400,111420,2238.68,2240.36,that's essentially poly-A tail.,"[51187, 326, 338, 6986, 7514, 12, 32, 7894, 13...",0.0,-0.220976,1.672489,0.004981,1
536,401,111420,2240.36,2241.52,"So that already tells you,","[51271, 1406, 326, 1541, 4952, 345, 11, 51329]",0.0,-0.220976,1.672489,0.004981,1
537,402,111420,2241.52,2245.16,"even if these labels are missing,","[51329, 772, 611, 777, 14722, 389, 4814, 11, 5...",0.0,-0.220976,1.672489,0.004981,1
538,403,111420,2245.16,2246.32,you know that the...,"[51511, 345, 760, 326, 262, 986, 51569]",0.0,-0.220976,1.672489,0.004981,1
539,0,0,2226.00,2230.32,"this five prime capping, this sort of a squar...","[50363, 428, 1936, 6994, 1275, 2105, 11, 428, ...",0.0,-0.349091,1.596639,0.241382,2


In [53]:
subclip_0_df.sort_values(by=["start"]).iloc[528:545]

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,subclip_index
528,393,111420,2222.24,2224.44,they need to be processed and mature.,"[50365, 484, 761, 284, 307, 13686, 290, 15345,...",0.0,-0.220976,1.672489,0.004981,1
529,394,111420,2224.44,2226.72,And this maturation process requires,"[50475, 843, 428, 285, 36921, 1429, 4433, 50589]",0.0,-0.220976,1.672489,0.004981,1
539,0,0,2226.00,2230.32,"this five prime capping, this sort of a squar...","[50363, 428, 1936, 6994, 1275, 2105, 11, 428, ...",0.0,-0.349091,1.596639,0.241382,2
530,395,111420,2226.72,2230.44,"this five prime capping, this sort of a squar...","[50589, 428, 1936, 6994, 1275, 2105, 11, 428, ...",0.0,-0.220976,1.672489,0.004981,1
540,1,0,2230.32,2232.40,and then poly a tail tagging.,"[50579, 290, 788, 7514, 257, 7894, 49620, 13, ...",0.0,-0.349091,1.596639,0.241382,2
531,396,111420,2230.44,2232.52,and then poly-A tail tagging.,"[50775, 290, 788, 7514, 12, 32, 7894, 49620, 1...",0.0,-0.220976,1.672489,0.004981,1
541,2,0,2232.40,2233.60,"So maybe on the footnote,","[50683, 1406, 3863, 319, 262, 38512, 11, 50743]",0.0,-0.349091,1.596639,0.241382,2
532,397,111420,2232.52,2233.72,"So maybe on the footnote,","[50879, 1406, 3863, 319, 262, 38512, 11, 50939]",0.0,-0.220976,1.672489,0.004981,1
542,3,0,2233.60,2236.20,I mentioned five prime cap is a purple square.,"[50743, 314, 4750, 1936, 6994, 1451, 318, 257,...",0.0,-0.349091,1.596639,0.241382,2
533,398,111420,2233.72,2236.32,"I mentioned five prime cap is a purple square,","[50939, 314, 4750, 1936, 6994, 1451, 318, 257,...",0.0,-0.220976,1.672489,0.004981,1


In [54]:
subclip_0_df = subclip_0_df.sort_values(by=["start"])
subclip_0_df = subclip_0_df.reset_index(drop=True)

subclip_0_df["start_diff"] = subclip_0_df["start"].diff()
subclip_0_df["start_diff"].iloc[0] = 0

/var/folders/xd/10nyn_j13qs58rww11sjkgy80000gn/T/ipykernel_98810/1052025233.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subclip_0_df["start_diff"].iloc[0] = 0


In [55]:
subclip_0_df.iloc[528:545]

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,subclip_index,start_diff
528,393,111420,2222.24,2224.44,they need to be processed and mature.,"[50365, 484, 761, 284, 307, 13686, 290, 15345,...",0.0,-0.220976,1.672489,0.004981,1,2.56
529,394,111420,2224.44,2226.72,And this maturation process requires,"[50475, 843, 428, 285, 36921, 1429, 4433, 50589]",0.0,-0.220976,1.672489,0.004981,1,2.20
530,0,0,2226.00,2230.32,"this five prime capping, this sort of a squar...","[50363, 428, 1936, 6994, 1275, 2105, 11, 428, ...",0.0,-0.349091,1.596639,0.241382,2,1.56
531,395,111420,2226.72,2230.44,"this five prime capping, this sort of a squar...","[50589, 428, 1936, 6994, 1275, 2105, 11, 428, ...",0.0,-0.220976,1.672489,0.004981,1,0.72
532,1,0,2230.32,2232.40,and then poly a tail tagging.,"[50579, 290, 788, 7514, 257, 7894, 49620, 13, ...",0.0,-0.349091,1.596639,0.241382,2,3.60
533,396,111420,2230.44,2232.52,and then poly-A tail tagging.,"[50775, 290, 788, 7514, 12, 32, 7894, 49620, 1...",0.0,-0.220976,1.672489,0.004981,1,0.12
534,2,0,2232.40,2233.60,"So maybe on the footnote,","[50683, 1406, 3863, 319, 262, 38512, 11, 50743]",0.0,-0.349091,1.596639,0.241382,2,1.96
535,397,111420,2232.52,2233.72,"So maybe on the footnote,","[50879, 1406, 3863, 319, 262, 38512, 11, 50939]",0.0,-0.220976,1.672489,0.004981,1,0.12
536,3,0,2233.60,2236.20,I mentioned five prime cap is a purple square.,"[50743, 314, 4750, 1936, 6994, 1451, 318, 257,...",0.0,-0.349091,1.596639,0.241382,2,1.08
537,398,111420,2233.72,2236.32,"I mentioned five prime cap is a purple square,","[50939, 314, 4750, 1936, 6994, 1451, 318, 257,...",0.0,-0.220976,1.672489,0.004981,1,0.12


In [56]:
subclip_0_df[
    (subclip_0_df["start"] > 2226.00) & (subclip_0_df["subclip_index"] < 2)
].index

Index([531, 533, 535, 537, 539, 541, 543, 545, 547], dtype='int64')

In [57]:
subclip_0_df[(subclip_0_df["subclip_index"] == 2)]["start"].min()

2226.0

In [58]:
# Drop every row where start > 2226.00 and subclip_index < 1


for i in range(1, len(transcription)):
    min_start_i = subclip_0_df[(subclip_0_df["subclip_index"] == i)]["start"].min()

    subclip_0_df = subclip_0_df.drop(
        subclip_0_df[
            (subclip_0_df["start"] > min_start_i) & (subclip_0_df["subclip_index"] < i)
        ].index
    )

In [59]:
subclip_0_df[133:145]

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob,subclip_index,start_diff
133,133,109600,1097.00,1106.00,I said coded manner because which amino acid ...,"[50413, 314, 531, 30817, 5642, 780, 543, 23206...",0.0,-0.154138,1.660232,0.238554,0,24.00
134,134,109600,1108.00,1125.00,How do you think this position happens? So mR...,"[50963, 1374, 466, 345, 892, 428, 2292, 4325, ...",0.0,-0.154138,1.660232,0.238554,0,11.00
135,0,0,1108.00,1110.24,How do you think this position happens?,"[50363, 1374, 466, 345, 892, 428, 2292, 4325, ...",0.0,-0.299951,1.581395,0.019813,1,0.00
136,1,0,1110.24,1112.96,So mRNA will have different nucleoside fields,"[50475, 1406, 47227, 481, 423, 1180, 17751, 41...",0.0,-0.299951,1.581395,0.019813,1,2.24
137,2,0,1112.96,1116.92,and I could have tRNA bringing in one amino a...,"[50611, 290, 314, 714, 423, 256, 27204, 6079, ...",0.0,-0.299951,1.581395,0.019813,1,2.72
138,3,0,1116.92,1119.02,"How do I know how does the system,","[50809, 1374, 466, 314, 760, 703, 857, 262, 10...",0.0,-0.299951,1.581395,0.019813,1,3.96
139,4,0,1119.02,1121.20,"how can you encode for the fact that,","[50914, 703, 460, 345, 37773, 329, 262, 1109, ...",0.0,-0.299951,1.581395,0.019813,1,2.10
140,5,0,1122.42,1124.52,"if I hit this particular code off,","[51084, 611, 314, 2277, 428, 1948, 2438, 572, ...",0.0,-0.299951,1.581395,0.019813,1,3.40
141,6,0,1124.52,1126.74,I will bring in the correct amino acid.,"[51189, 314, 481, 2222, 287, 262, 3376, 23206,...",0.0,-0.299951,1.581395,0.019813,1,2.10
142,7,0,1128.36,1129.60,There's chemistry behind it.,"[51381, 1318, 338, 16585, 2157, 340, 13, 51443]",0.0,-0.299951,1.581395,0.019813,1,3.84


In [62]:
subclip_0_df["text"].to_list()

[' Thank you all.',
 ' Good afternoon.',
 ' How are you guys doing?',
 ' It looks fun.',
 ' How many fans have done themselves last week?',
 ' Too easy?',
 ' At least, maybe you think, well, applied the concepts that we learned in the past.',
 ' I hope that we all have relevant problems, for modern students like you guys.',
 ' At least, I hope you had some fun from doing it and not finding it too challenging.',
 ' The final exam questions, they will be largely in the context of problem solving like this.',
 ' And the level of challenge will be suddenly a little bit less, perhaps,',
 ' because in the final exam you are kind of limited on it.',
 ' So the highest level of challenge you will see, and at least my point in the course,',
 ' will be represented by the problem sets questions.',
 ' So if you feel that we understand them and you can tackle them, especially after the session,',
 ' and you are engaged in the class and you really should not need to worry suddenly from the',
 " three

In [63]:
full_text = " ".join(subclip_0_df["text"].to_list())

In [64]:
full_text

" Thank you all.  Good afternoon.  How are you guys doing?  It looks fun.  How many fans have done themselves last week?  Too easy?  At least, maybe you think, well, applied the concepts that we learned in the past.  I hope that we all have relevant problems, for modern students like you guys.  At least, I hope you had some fun from doing it and not finding it too challenging.  The final exam questions, they will be largely in the context of problem solving like this.  And the level of challenge will be suddenly a little bit less, perhaps,  because in the final exam you are kind of limited on it.  So the highest level of challenge you will see, and at least my point in the course,  will be represented by the problem sets questions.  So if you feel that we understand them and you can tackle them, especially after the session,  and you are engaged in the class and you really should not need to worry suddenly from the  three lecture series that I'm very happy to provide here for this cour

In [74]:
detected_frames = pd.read_csv("output/detected_frames.txt", sep=" ", header=None)

In [75]:
detected_frames

,0,1,2,3,4,5
0,frame:0,NaN,NaN,NaN,pts:1321984,pts_time:103.28
1,lavfi.scene_score=0.140596,NaN,NaN,NaN,NaN,NaN
2,frame:1,NaN,NaN,NaN,pts:1323520,pts_time:103.4
3,lavfi.scene_score=0.221395,NaN,NaN,NaN,NaN,NaN
4,frame:2,NaN,NaN,NaN,pts:1325568,pts_time:103.56
...,...,...,...,...,...,...
61,lavfi.scene_score=0.145158,NaN,NaN,NaN,NaN,NaN
62,frame:31,NaN,NaN,pts:61705216,pts_time:4820.72,NaN
63,lavfi.scene_score=0.107849,NaN,NaN,NaN,NaN,NaN
64,frame:32,NaN,NaN,pts:70086656,pts_time:5475.52,NaN


In [77]:
detected_frames.drop(columns=[1, 2, 3, 4])  # , inplace=True)

,0,5
0,frame:0,pts_time:103.28
1,lavfi.scene_score=0.140596,NaN
2,frame:1,pts_time:103.4
3,lavfi.scene_score=0.221395,NaN
4,frame:2,pts_time:103.56
...,...,...
61,lavfi.scene_score=0.145158,NaN
62,frame:31,NaN
63,lavfi.scene_score=0.107849,NaN
64,frame:32,NaN


In [85]:
import re

# Read the text file
with open("output/detected_frames.txt", "r") as file:
    content = file.read()

# Extract frame numbers and pts_time numbers using regular expressions
frame_numbers = re.findall(r"frame:(\d+)", content)
# pts_time_numbers = re.findall(r'pts_time:(\d+\.\d+)', content)
pts_time_numbers = re.findall(r"pts_time:(\d+)", content)

# Print the extracted numbers
print("Frame Numbers:")
print(frame_numbers)

print("\npts_time Numbers:")
print(pts_time_numbers)

Frame Numbers:
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32']

pts_time Numbers:
['103', '103', '103', '103', '103', '104', '104', '105', '146', '261', '383', '731', '985', '1239', '1341', '1374', '1706', '2000', '2002', '2008', '3013', '3018', '3092', '3282', '3807', '4542', '4554', '4769', '4770', '4803', '4805', '4820', '5475']


In [86]:
# Convert the lists to dictionariers
frame_dict = dict(zip(frame_numbers, pts_time_numbers))

In [87]:
print(len(frame_dict))
print(len(frame_numbers))
print(len(pts_time_numbers))

33
33
33


In [81]:
frame_dict

{'0': '103.28',
 '1': '103.4',
 '2': '103.56',
 '3': '103.68',
 '4': '103.84',
 '5': '104.16',
 '6': '105.04',
 '7': '146.36',
 '8': '261.28',
 '9': '383.72',
 '10': '985.32',
 '11': '1239.04',
 '12': '1341.68',
 '13': '1374.44',
 '14': '1706.2',
 '15': '2000.64',
 '16': '2002.12',
 '17': '2008.44',
 '18': '3018.6',
 '19': '3092.6',
 '20': '3282.36',
 '21': '3807.24',
 '22': '4554.12',
 '23': '4769.8',
 '24': '4770.48',
 '25': '4803.88',
 '26': '4805.24',
 '27': '4820.72',
 '28': '5475.52'}